Скрит позволяет разбивать изображения на части для обучения

In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# Параметры разбиения
PATCH_SIZE = 640
OVERLAP = 5
OUTPUT_DIR = "../DataSet/SplitData/Test2/"
DATA_DIR = "../DataSet/MyData/Test/"


In [2]:
def split_image_and_labels(image_path, label_path, patch_size, overlap, output_dir):
    image = cv2.imread(image_path)
    h, w, _ = image.shape
    step = patch_size - overlap

    os.makedirs(os.path.join(output_dir, "images"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "labels"), exist_ok=True)

    # метки
    labels = []
    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            for line in f.readlines():
                labels.append([float(x) for x in line.strip().split()])

    # разрезаем
    idx = 0
    for y in range(0, h, step):
        for x in range(0, w, step):
            # координаты части изображения
            x_end = min(x + patch_size, w)
            y_end = min(y + patch_size, h)
            patch = image[y:y_end, x:x_end]
            patch_h, patch_w, _ = patch.shape

            # Генерируем новое имя для части
            img_name = f"{os.path.splitext(os.path.basename(image_path))[0]}_{idx}.jpg"
            lbl_name = f"{os.path.splitext(os.path.basename(label_path))[0]}_{idx}.txt"

            cv2.imwrite(os.path.join(output_dir, "images", img_name), patch)

            # метки для этой части
            patch_labels = []
            for label in labels:
                cls, x_center, y_center, width, height = label
                x_abs = x_center * w
                y_abs = y_center * h
                box_w = width * w
                box_h = height * h

                # проверяем попадание метки в текущую область
                if x < x_abs < x_end and y < y_abs < y_end:
                    new_x_center = (x_abs - x) / patch_w
                    new_y_center = (y_abs - y) / patch_h
                    new_width = box_w / patch_w
                    new_height = box_h / patch_h

                    patch_labels.append([cls, new_x_center, new_y_center, new_width, new_height])

            with open(os.path.join(output_dir, "labels", lbl_name), "w") as f:
                for lbl in patch_labels:
                    f.write(" ".join(map(str, lbl)) + "\n")

            idx += 1

In [3]:

# Обработка всего набора данных

for img_name in tqdm(os.listdir(os.path.join(DATA_DIR, "images"))):
    img_path = os.path.join(DATA_DIR, "images", img_name)
    label_path = os.path.join(DATA_DIR, "labels", f"{os.path.splitext(img_name)[0]}.txt")

    split_image_and_labels(
        image_path=img_path,
        label_path=label_path,
        patch_size=PATCH_SIZE,
        overlap=OVERLAP,
        output_dir=OUTPUT_DIR,
    )

  0%|          | 0/138 [00:00<?, ?it/s]

100%|██████████| 138/138 [01:05<00:00,  2.11it/s]
